In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_sale_raw = pd.read_csv('1-sale-raw.csv')
df_rent_raw = pd.read_csv('1-rent-raw.csv')

# Make a back up copy:

In [3]:
df_sale = df_sale_raw.copy()

In [4]:
df_rent = df_rent_raw.copy()

# Give it a look

In [5]:
df_sale.head(3)

,link,price,title,location,post-date,bedrooms,bathrooms,receptions
0,https://www.zoopla.co.uk//for-sale/details/602...,"£170,000",2 bed terraced house for sale,"Spring Hill, Birmingham B24",Listed on 19th Nov 2021,2.0,1.0,2.0
1,https://www.zoopla.co.uk//for-sale/details/602...,"£275,000",3 bed semi-detached house for sale,"Quinton Lane, Quinton, Birmingham B32",Listed on 18th Nov 2021,3.0,NaN,NaN
2,https://www.zoopla.co.uk//for-sale/details/602...,"£250,000",3 bed semi-detached house for sale,"Loynells Road, Rednal, Birmingham B45",Listed on 18th Nov 2021,3.0,1.0,2.0


In [6]:
df_sale.shape

(3290, 8)

In [7]:
df_rent.head(3)

,link,price,title,location,post-date,bedrooms,bathrooms,receptions
0,https://www.zoopla.co.uk//to-rent/details/6022...,"£1,000 pcm",3 bed semi-detached house to rent,"Ascot Road, Birmingham 9Pb B13",Listed on 19th Nov 2021,3.0,1.0,NaN
1,https://www.zoopla.co.uk//to-rent/details/5295...,"£1,100 pcm",2 bed flat to rent,"Dayus House, Tenby Street South, Jewellery Qua...",Listed on 18th Nov 2021,2.0,1.0,1.0
2,https://www.zoopla.co.uk//to-rent/details/5120...,"£1,595 pcm",3 bed terraced house to rent,"Ravenhurst Road, Harborne, Birmingham B17",Listed on 18th Nov 2021,3.0,NaN,NaN


In [8]:
df_rent.shape

(2612, 8)


# Fix the price :


In [9]:
df_sale.price.head(3)

0    £170,000
1    £275,000
2    £250,000
Name: price, dtype: object

In [10]:
df_sale.price = df_sale.price.apply(lambda x: x.replace('£',''))
df_sale.price = df_sale.price.apply(lambda x: x.replace(',',''))

In [11]:
df_sale.price.astype(int)

ValueError: invalid literal for int() with base 10: 'POA'

In [12]:
# Lets figure out what just happened

In [13]:
for x in df_sale.price:
    try:
        int(x)
    except:
        print(x)

POA
POA
POA
POA
POA
POA
POA
POA
POA
POA
POA
POA
POA


In [14]:
# Good, these rows aren't helpful for our analysis
# let's convert them to NAN and drop them.
df_sale.price.replace('POA', np.nan, inplace=True)
df_sale.dropna(subset=['price'], inplace=True)

In [15]:
df_sale.price = df_sale.price.astype(int)
df_sale.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3277 entries, 0 to 3289
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   link        3277 non-null   object 
 1   price       3277 non-null   int32  
 2   title       3277 non-null   object 
 3   location    3277 non-null   object 
 4   post-date   3277 non-null   object 
 5   bedrooms    3213 non-null   float64
 6   bathrooms   2995 non-null   float64
 7   receptions  2743 non-null   float64
dtypes: float64(3), int32(1), object(4)
memory usage: 217.6+ KB


# Split the location
We are only interested in postcodes.

In [16]:
def get_postcode(x):
    try:
        return re.findall(r'B\d?\d', x)[-1]
    except:
        return np.nan

In [17]:
df_sale['postcode'] = df_sale.location.apply(get_postcode)
df_sale['postcode'].value_counts()

B1     367
B5     234
B16    188
B15    178
B12    160
B17    132
B13    103
B3     102
B26    100
B31     98
B4      88
B36     76
B23     75
B28     70
B18     69
B29     68
B73     65
B14     57
B66     57
B42     57
B20     56
B8      54
B32     53
B30     52
B24     52
B21     49
B72     46
B67     43
B38     40
B45     37
B27     34
B43     34
B37     33
B33     33
B25     32
B11     28
B34     28
B92     27
B6      26
B10     26
B2      24
B19     21
B9      21
B44     17
B90     16
B35     13
B76     12
B47     11
B7       4
B48      3
B68      3
B46      3
B62      1
Name: postcode, dtype: int64

# Let's make our dataframe ready

In [18]:
df_sale.columns

Index(['link', 'price', 'title', 'location', 'post-date', 'bedrooms',
       'bathrooms', 'receptions', 'postcode'],
      dtype='object')

In [19]:
df_sale[df_sale.postcode.isnull()]

,link,price,title,location,post-date,bedrooms,bathrooms,receptions,postcode
1702,https://www.zoopla.co.uk//for-sale/details/595...,600000,1 bed flat for sale,"Turnberry Quay, London E14",Listed on 1st Sep 2021,1.0,1.0,1.0,NaN


In [20]:
# That's an error from the website results...
# E14 refers to London..
# Let's get rid of it..

In [21]:
df_sale.dropna(subset=['postcode'], inplace=True)

# Type of properties

In [22]:
df_sale['type'] = df_sale.title.apply(lambda x: x.replace(''.join(re.findall('\d+\sbed\s', x)), '').lower())
df_sale['type'] = df_sale.type.apply(lambda x: x.replace(''.join(re.findall('\sfor\ssale', x)), '').lower())
df_sale['type'].unique()

array(['terraced house', 'semi-detached house', 'flat', 'property',
       'detached bungalow', 'detached house', 'end terrace house',
       'semi-detached bungalow', 'town house', 'maisonette',
       'link-detached house', 'bungalow', 'mews house', 'parking/garage',
       'studio', 'block of flats', 'barn conversion', 'terraced bungalow',
       'penthouse', 'room', 'cottage', 'lodge', 'land'], dtype=object)

In [23]:
df_sale

,link,price,title,location,post-date,bedrooms,bathrooms,receptions,postcode,type
0,https://www.zoopla.co.uk//for-sale/details/602...,170000,2 bed terraced house for sale,"Spring Hill, Birmingham B24",Listed on 19th Nov 2021,2.0,1.0,2.0,B24,terraced house
1,https://www.zoopla.co.uk//for-sale/details/602...,275000,3 bed semi-detached house for sale,"Quinton Lane, Quinton, Birmingham B32",Listed on 18th Nov 2021,3.0,NaN,NaN,B32,semi-detached house
2,https://www.zoopla.co.uk//for-sale/details/602...,250000,3 bed semi-detached house for sale,"Loynells Road, Rednal, Birmingham B45",Listed on 18th Nov 2021,3.0,1.0,2.0,B45,semi-detached house
3,https://www.zoopla.co.uk//for-sale/details/602...,239000,2 bed flat for sale,"Heritage Court, 15 Warstone Lane, Birmingham B18",Listed on 18th Nov 2021,2.0,2.0,1.0,B18,flat
4,https://www.zoopla.co.uk//for-sale/details/602...,240000,3 bed semi-detached house for sale,"Old Town Close, Kings Norton, Birmingham B38",Listed on 18th Nov 2021,3.0,1.0,1.0,B38,semi-detached house
...,...,...,...,...,...,...,...,...,...,...
3285,https://www.zoopla.co.uk//for-sale/details/475...,300000,2 bed property for sale,"Hagley Road, Edgbaston, Birmingham B17",Listed on 19th Oct 2017,2.0,1.0,1.0,B17,property
3286,https://www.zoopla.co.uk//for-sale/details/451...,115950,1 bed flat for sale,"42 Ryland St, Birmingham B16",Listed on 4th Oct 2017,1.0,1.0,1.0,B16,flat
3287,https://www.zoopla.co.uk//for-sale/details/446...,180000,4 bed end terrace house for sale,"Abbey Road, Erdington, Birmingham, West Midlan...",Listed on 10th Aug 2017,4.0,2.0,2.0,B23,end terrace house
3288,https://www.zoopla.co.uk//for-sale/details/428...,319950,2 bed flat for sale,"New St, Birmingham B2",Listed on 6th Feb 2017,2.0,2.0,1.0,B2,flat


# Let's repeat them all to df_rent:

In [24]:
def fix_rent_price(x):
    x = x.replace('£','')
    x = x.replace(' pcm','')
    x = x.replace(',','')
    return x
df_rent.price = df_rent.price.apply(fix_rent_price)
df_rent.price.replace('POA', np.nan, inplace=True)
df_rent.dropna(subset=['price'], inplace=True)
df_rent.price = df_rent.price.astype(int)
def no_of_beds_is_na(x):
    try:
        if int(x[0]):
            return False
    except:
        return True
df_rent['type'] = df_rent.title.apply(lambda x: x.replace(''.join(re.findall('\d+\sbed\s', x)), '').lower())
df_rent['type'] = df_rent.type.apply(lambda x: x.replace(' to rent',''))
def get_postcode(x):
    try:
        return re.findall(r'B\d?\d', x)[-1]
    except:
        return np.nan
df_rent['postcode'] = df_rent.location.apply(get_postcode)
df_rent.dropna(subset=['postcode'], inplace=True)

In [25]:
df_rent.head()

,link,price,title,location,post-date,bedrooms,bathrooms,receptions,type,postcode
0,https://www.zoopla.co.uk//to-rent/details/6022...,1000,3 bed semi-detached house to rent,"Ascot Road, Birmingham 9Pb B13",Listed on 19th Nov 2021,3.0,1.0,NaN,semi-detached house,B13
1,https://www.zoopla.co.uk//to-rent/details/5295...,1100,2 bed flat to rent,"Dayus House, Tenby Street South, Jewellery Qua...",Listed on 18th Nov 2021,2.0,1.0,1.0,flat,B1
2,https://www.zoopla.co.uk//to-rent/details/5120...,1595,3 bed terraced house to rent,"Ravenhurst Road, Harborne, Birmingham B17",Listed on 18th Nov 2021,3.0,NaN,NaN,terraced house,B17
3,https://www.zoopla.co.uk//to-rent/details/5357...,750,1 bed flat to rent,"Albion House, Pope Street, Jewellery Quarter B1",Listed on 18th Nov 2021,1.0,1.0,1.0,flat,B1
4,https://www.zoopla.co.uk//to-rent/details/6022...,875,3 bed property to rent,"Tyndale Crescent, Birmingham B43",Listed on 18th Nov 2021,3.0,1.0,1.0,property,B43


In [26]:
df_sale.to_csv('2-sale-cleaned.csv', index=False)
df_rent.to_csv('2-rent-cleaned.csv', index=False)